In [ ]:
import geopandas
import pandas as pd
import pgeocode
import plotly.graph_objects as go
nomi = pgeocode.Nominatim('ca')
import numpy as np
import folium
try:
    import haversine as hs
except:
    !pip install --user haversine
    import haversine as hs

In [ ]:
data = pd.read_csv('data/Connected member addresses for network map November 2022 - Sheet1.csv', header=None, 
                   names=['Member', 'Address', 'City', 'Postal Code'])
data.iloc[56]['Postal Code'] = 'T0L 0Z0' # Someone entered this wihout a space
data.drop([36, 38], axis=0, inplace=True) # Centers in Germany and France
data

In [ ]:
len(data['City'].unique())

In [ ]:
lats = []
lons = []
for i in data['Postal Code']:
    location = nomi.query_postal_code(i)
    lats.append(location['latitude'])
    lons.append(location['longitude'])
data['Latitude'] = lats
data['Longitude'] = lons
data['geometry'] = geopandas.points_from_xy(data['Latitude'], data['Longitude'])
data.sort_values(by='Latitude', inplace=True)
data

In [ ]:
# it looks like it is returning the wrong location for Morley, it should be 51.16153, -114.85095
data[data['City'] == 'Morley']

In [ ]:
# def getCityLocation(city):
#     location = data[data['City'] == city]#[['Latitude', 'Longitude']].mean()
#     #location = nomi.query_location(city)
#     #if not location['latitude'].mean() > 40:
#     #    postal_code = data[data['City'] == city]['Postal Code'].unique()[0]
#     #    location = nomi.query_postal_code(postal_code)
#     return location['Latitude'].mean(), location['Longitude'].mean()

# cities_dict = {}
# for city in data['City'].unique():
#     cities_dict[city] = getCityLocation(city)
# cities = pd.DataFrame.from_dict(cities_dict, orient='index', columns=['Latitude', 'Longitude'])
# cities.reset_index(inplace=True)
# cities.rename(columns={'index':'Name'}, inplace=True)
# cities

In [ ]:
#cities.to_csv('data/cities.csv', index=False)

The below section simply recalculates the lat/long coords using a different implementation of Nominatim, dropping the point in the center of the population centers.

Manually inspecting all the points, the following are still a little off:

- Wainwright
- Taber
- Barrhead

With just three, it's simpler to manually fix them using the coordinates given by Wikipedia/GeoHack

In [ ]:
cities = pd.DataFrame(data['City'].unique(), columns=['Name'])

from geopy.geocoders import Nominatim
nomi2 = Nominatim(user_agent='my_application')

lats2 = []
lons2 = []
    
for c in cities['Name']:
    location2 = nomi2.geocode(c + ', AB') # catches a few edge cases of similarly named places outside of AB
    lats2.append(location2[1][0])
    lons2.append(location2[1][1])
cities['Latitude'] = lats2
cities['Longitude'] = lons2

# Taber
cities.at[1,'Latitude'] = 49.784722
cities.at[1,'Longitude'] = -112.150833

# Wainwright
cities.at[18,'Latitude'] = 52.833333
cities.at[18,'Longitude'] = -110.866667

# Barrhead
cities.at[30,'Latitude'] = 54.116667
cities.at[30,'Longitude'] = -114.4

    
cities

In [ ]:
cities_map = folium.Map(location=[cities['Latitude'].mean(), cities['Longitude'].mean()], zoom_start=5)
for i in range(0,len(cities)):
    folium.Marker([cities.iloc[i]['Latitude'], cities.iloc[i]['Longitude']], popup=cities.iloc[i]['Name']).add_to(cities_map)
cities_map

In [ ]:
cities.sort_values(by='Latitude', inplace=True)

distances_list = []
for i in range(len(cities)):
    distances_list.append([cities.iloc[i]['Name'], cities.iloc[i-1]['Name'], 
                      hs.haversine((cities.iloc[i]['Latitude'], cities.iloc[i]['Longitude']), 
                                   (cities.iloc[i-1]['Latitude'], cities.iloc[i-1]['Longitude']))])
dbc = pd.DataFrame(distances_list, columns=['City', 'Next City', 'Distance'])
dbc

In [ ]:
dbc[dbc['Distance'] < 75]

In [ ]:
cities_to_replace = {
    'Taber':'Lethbridge',
    'Dunmore':'Medicine Hat',
    'Morley':'Canmore',
    'Banff':'Canmore',
    'Ponoka':'Lacombe',
    'Wetaskiwin':'Camrose',
    'Leduc':'Edmonton',
    'Nisku':'Edmonton',
    'Spruce Grove':'Edmonton',
    'Stony Plain':'Edmonton',
    'St. Albert':'Edmonton',
    'Morinville':'Edmonton',
    'Elk Point':'Lac La Biche',
    'Grouard':'High Prairie',
    'Red Earth Creek':'High Prairie',
    'Fort Vermilion':'Grimshaw',
}

In [ ]:
rows_to_drop = []
for i in range(len(cities)):
    if cities.iloc[i]['Name'] in cities_to_replace.keys():
        print(i, cities.iloc[i]['Name'])
        rows_to_drop.append(i)
rows_to_drop

In [ ]:
updated_cities = cities.drop(rows_to_drop, axis=0)
updated_cities.reset_index(inplace=True, drop=True)
updated_cities

In [ ]:
#updated_cities.to_csv('data/updated_cities.csv', index=False)

In [ ]:
updated_cities_map = folium.Map(location=[updated_cities['Latitude'].mean(), updated_cities['Longitude'].mean()], zoom_start=5)
for i in range(0,len(updated_cities)):
    folium.Marker([updated_cities.iloc[i]['Latitude'], updated_cities.iloc[i]['Longitude']], popup=updated_cities.iloc[i]['Name']).add_to(updated_cities_map)
updated_cities_map

In [ ]:
fig0 = go.Figure(data=go.Scattergeo(
    lat = data['Latitude'],
    lon = data['Longitude'],
    text = data['Member']
)
               )

fig0.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        #landcolor = "rgb(212, 212, 212)",
        landcolor = "rgb(255, 255, 255)",
        subunitcolor = "rgb(0, 0, 0)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig0.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
fig0.show()
#fig.write_html('membermap.html')

In [ ]:
fig3 = go.Figure(data=go.Scattergeo(
    lat = updated_cities['Latitude'],
    lon = updated_cities['Longitude'],
    text = updated_cities['Name']
)
               )

fig3.update_layout(
    geo = dict(
        scope = 'north america',
        showland = True,
        #landcolor = "rgb(212, 212, 212)",
        landcolor = "rgb(255, 255, 255)",
        subunitcolor = "rgb(0, 0, 0)",
        countrycolor = "rgb(255, 255, 255)",
        showlakes = False,
        lakecolor = "rgb(255, 255, 255)",
        showsubunits = True,
        showcountries = True,
        resolution = 50,
        projection = dict(
            type = 'conic conformal',
            rotation_lon = -100
        ),
        lonaxis = dict(
            showgrid = True,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = True,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    title='Cybera Member locations',
    height=600,
    width=600,
    margin={"r":0,"t":0,"l":0,"b":0}
)
fig3.update_geos(lataxis_range=[48,61],
               lonaxis_range=[-120,-110])
fig3.show()
#fig3.write_html('membermap.html')

In [ ]:
#!pip install kaleido
# fig3.write_image('membermap.svg')